## Make the GPT-2 simulate republican cases through Reinforcement Learning

In this homework, we will first train a reward model that assign higher reward to documents that sounds more like republican cases. Then we use RL to guide GPT-2 to complete the democratric cases in a republican way.

The reward model is covered in a previous notebook. All TODOs you need to finish lie in the RL part.

In [ ]:
#!pip install transformers trl
#pip install google-colab


In [1]:
# load sc_cases_cleaned.pkl that we used in the previous notebooks
#!wget https://github.com/elliottash/nlp_lss_2023/blob/master/notebooks/sc_cases_cleaned.pkl
from google.colab import files                                                                                                                                                                                       
uploaded = files.upload()

Saving sc_cases_cleaned.pkl to sc_cases_cleaned.pkl


In [7]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer, DistilBertTokenizerFast, DistilBertForSequenceClassification

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler


### train a classification model as our reward model

In [8]:
#!pip install trl

In [10]:
df = pd.read_pickle('sc_cases_cleaned.pkl', compression='gzip')
df = df.assign(author_id=(df['authorship']).astype('category').cat.codes)

# gpu or cpu?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder 
model = DistilBertForSequenceClassification.from_pretrained(model_name)

tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
inputs = tokenizer(df['opinion_text'].tolist(), return_tensors="pt", padding=True, truncation=True)
labels = torch.tensor(df['x_republican'].tolist()).long()

optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},  
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['opinion_text'].tolist(), df['x_republican'].tolist(), test_size=.2)

# generate batches
X_train, X_test, y_train, y_test = np.array(X_train[:608]), np.array(X_test[:152]), np.array(y_train[:608]), np.array(y_test[:152])
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train, X_test, y_train, y_test = X_train.reshape(-1, 8), X_test.reshape(-1, 8), y_train.reshape(-1, 8), y_test.reshape(-1, 8)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train, X_test = X_train.tolist(), X_test.tolist()

# train
from tqdm import tqdm

model.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for text, labels in tqdm(zip(X_train, y_train), total=len(X_train)):
        # prepare model input through our tokenizer
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
        # place everything on the right device
        model_inputs = {k:v.to(device) for k,v in model_inputs.items()}
        # labels have to be torch long tensors
        labels = torch.tensor(labels).long().to(device)
        # now, we can perform the forward pass
        output = model(**model_inputs, labels=labels)
        loss, logits = output[:2]
        # and the backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

torch.save(model, 'republican_classifier.pt')
republican_classifier = model

cuda


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

(608,) (152,) (608,) (152,)
(76, 8) (19, 8) (76, 8) (19, 8)


100%|██████████| 76/76 [00:16<00:00,  4.68it/s]


In [11]:
from transformers import DistilBertTokenizer, TextClassificationPipeline

distil_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', max_length=512, truncation=True)
pipeline = TextClassificationPipeline(model=republican_classifier, tokenizer=distil_tokenizer, return_all_scores=True, device=republican_classifier.device)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [12]:
#!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 7.3 MB/s eta 0:00:00


In [13]:
texts = df.loc[df['x_republican'] == 0, 'opinion_text'].tolist()
texts[0]

'JUSTICE GINSBURG delivered the opinion of the Court.\n\n A motion by a federal prisoner for postconviction relief under 28 U.S.C. § 2255 is subject to a one-year time limitation that generally runs from "the date on which the judgment of conviction becomes final." § 2255, P6(1). This case concerns the starting date for the one-year limitation. It presents a narrow but recurring question on which courts of appeals have divided: When a defendant in a federal prosecution takes an unsuccessful direct appeal from a judgment of conviction, but does not next petition for a writ of certiorari from this Court, does the judgment become "final" for postconviction relief purposes (1) when the appellate court issues its mandate affirming the conviction, or, instead, (2) on the date, ordinarily 69 days later, when the time for filing a petition for certiorari expires?\n\nIn accord with this Court\'s consistent understanding of finality in the context of collateral review, and the weight of lower co

In [14]:
pipeline(X_test[0][0], truncation=True)

[[{'label': 'LABEL_0', 'score': 4.715484465123154e-05},
  {'label': 'LABEL_1', 'score': 0.9999527931213379}]]

### Reinforcement Learning

In [15]:
# prepare the dataset for reinforcement learning

from torch.utils.data import Dataset

class PPODataset(Dataset):
  def __init__(self, tokenizer, texts, input_min_text_length, input_max_text_length):
    self.tokenizer = tokenizer
    self.texts = texts
    self.random_sample = LengthSampler(input_min_text_length, input_max_text_length)

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, index):
    text = self.texts[index]
    sample = {}
    sample["input_ids"] = torch.tensor(tokenizer.encode(text)[: self.random_sample()])
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

dataset = PPODataset(tokenizer, texts, 10, 15)


In [16]:
# prepare arguments and configurations for RL

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

output_min_length = 50
output_max_length = 100
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

def collator(data):
  return dict((key, [d[key] for d in data]) for key in data[0])

config = PPOConfig(
    model_name="gpt2",
    learning_rate=1.41e-5,
    batch_size=32,
)

In [17]:
# TODO: prepare model (use gpt2), reference model, and PPO trainer for RL

#load gpt2
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

#add padding
tokenizer.pad_token = tokenizer.eos_token

#initialise ppo - model
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)


In [18]:
# TODO: conduct PPO training loop here. For efficiency, you can just train 3 batches.

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), total=len(ppo_trainer.dataloader)):
    if epoch >= 3:
      break

    #get queries from batch 
    query_tensors = batch["input_ids"]

    #response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    #use previous pipeline
    pipe_outputs = pipeline(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/5 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (5716 > 1024). Running this sequence through the model will result in indexing errors
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 60%|██████    | 3/5 [02:19<01:33, 46.62s/it]


In [19]:
# visualize the outcomes generated by the RL tuned GPT-2

#### get a batch from the dataset
game_data = dict()
game_data["query"] = [dataset[i]["query"] for i in range(15)]
query_tensors = [dataset[i]["input_ids"] for i in range(15)]

response_tensors_ref, response_tensors = [], []
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}
#### get response from gpt2 and gpt2_ref
for i in range(15):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(15)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(15)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in pipeline(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in pipeline(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,JUSTICE GINSBURG delivered the opinion of the ...,"Court. SOUTER, J., delivered the opinion of t...","Court. In O'Connor, C.J., joined by STEVENS, ...",-3.740855,-4.348896
1,Justice Breyer delivered the opinion of the Co...,entire conformity in various cases must be af...,trial court's acquiescence in the 'at the tim...,-4.116716,-4.090705
2,Justice Ginsburg delivered the opinion of the ...,There is enough indisputable evidence that the...,Holm's decisions now before the Court revealed...,-4.398184,-4.096229
3,Justice Ginsburg delivered the opinion of the ...,Breyer put in print the four propositions sep...,Ginsburg followed the majority's position in ...,-4.491657,-4.123387
4,Justice Breyer delivered the opinion of the Co...,"\nb\n\n1 The dealing (""calendar equation"") of ...",\nJUSTICE BLACKER dissented in part and joined...,-4.009022,-3.899661
5,Justice Ginsburg delivered the opinion of the ...,"Although the district court alone, excluding m...",There is nothing wrong with the police involve...,-4.471235,-4.291038
6,Justice Breyer delivered the opinion of the Co...,judgment applies wholly to the First Amendmen...,per op convention board of inquiry is and isn...,-4.530860,-3.705158
7,Justice Ginsburg delivered the opinion of the ...,opinion does not support the premise that U.S...,"is big news for Justice Ginsburg, author of t...",-4.092198,-4.081272
8,Justice Breyer delivered the opinion of the Co...,\nJUSTICE BRENNAN delivered the opinion of the...,"\nMalia Gennaro, acting for the Baja cartel, H...",-3.863258,-4.229008
9,Justice Ginsburg delivered the opinion of the ...,\nLaurence D. Bender writes – Click Here for t...,\nI\n\nThe Due Process Clause requires us to m...,-4.177264,-4.572536
